<a href="https://colab.research.google.com/github/Alouettesu/NewsTitler/blob/main/NewsTitler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Практическое задание: решение задачи обработки естественного языка
------

*Выполнил: Дианов Сергей Андреевич*

Бывало ли у вас такое, что, читая новостную ленту, вы видите интересный заголовок, а открыв его, обнаруживаете пустышку? Или текст, совершенно не соответствующий названию? Текст, в котором заявленная в заголовке тема не раскрыта никак? 

У меня такое тоже бывает, и особенно этим грешат журналисты mail.ru. А что если сделать машину, которая сама придумывает заголовки для новостей? Пусть на вход программе подаётся текст новости, а на выходе генерируется заголовок. Это я и выбрал в качестве своего первого задания по обработке естественного языка. Заголовки генерируются с помощью модели для библиотеки HuggingFace [IlyaGusev/rut5_base_headline_gen_telegram](https://huggingface.co/IlyaGusev/rut5_base_headline_gen_telegram). Новости загружаются с RSS-ленты сайта mail.ru.

Скрипт запрашивает содержимое RSS-файла и извлекает из него заголовки и ссылки на статьи. Затем загружает каждую веб-страницу по отдельности и извлекает из неё содержимое статьи путём нахождения html-тега `div class="article__text"`. Но если формат веб-страницы на сайте изменится, то потребуется доработка скрипта. Тем не менее, вы можете задать самостоятельно URL RSS-ленты и критерии поиска текста в форме ниже.

Результат работы выводится в стандартный вывод в форме таблицы. При запуске скрипта в какой-либо другой среде (не Google Colab) рекомендуется использовать терминал, который поддерживает регулировку ширины.

*Примечание: Иногда Mail.ru возвращает ошибку `HTTP Error 429 "Too many requests"`. Страницы, которые уже загрузились до появления этой ошибки, обрабатываются нормально.*

In [1]:
RSS_URL = "https://news.mail.ru/rss/main/66/" #@param {type:"string"}
HTML_ELEMENT = "div" #@param {type:"string"}
HTML_ATTRIB_NAME = "class" #@param ["class", "id"]
HTML_ATTRIB_VALUE = "article__text" #@param {type:"string"}

In [2]:
!pip install transformers sentencepiece sacremoses
!pip install feedparser
!pip install bs4
!pip install html2text
!pip install -U prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 1.3 MB 41.7 MB/s 
     |████████████████████████████████| 880 kB 51.9 MB/s 
     |████████████████████████████████| 120 kB 42.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ad0536439ddec65151a8aa187363cbb94b6a28dc95779629a2eaccd31f810ea1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.5 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=3dfa185224c6bc0f6b96481ec93f096790fda076042bed009b3a11236541c850
  Sto

In [3]:
# Импорт модулей для работы с текстом
import feedparser
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Импорт модулей для загрузки и извлечения текста из HTML
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import html2text

# Для вывода таблиц
from prettytable import PrettyTable

In [4]:
# Функция принимает на вход URL-адрес веб-страницы, загружает её содержимое,
# ищет текст новости, удаляет из него HTML-тэги, возвращает текст новости.

def getNewsText(news_url):
  req = Request(news_url)
  html_page = urlopen(req)

  soup = BeautifulSoup(html_page, "html.parser")

  txt = soup.find(HTML_ELEMENT, {HTML_ATTRIB_NAME : HTML_ATTRIB_VALUE})
  
  return txt.text


In [15]:
# Класс озаглавливателя новостей
# В конструкторе класса загружается модель и токенизатор.
# В методе Title на вход подаётся текст новости, на выходе выдаётся предложенный моделью заголовок.

class Titler:

  def __init__(self):
    self.model_name = "IlyaGusev/rut5_base_headline_gen_telegram"
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
    self.model = T5ForConditionalGeneration.from_pretrained(self.model_name)

  def Title(self, article_text):

    input_ids = self.tokenizer(
        article_text,
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"]

    output_ids = self.model.generate(
        input_ids=input_ids,
        max_new_tokens = 30
    )[0]
    headline = self.tokenizer.decode(output_ids, skip_special_tokens=True)
    return headline

In [ ]:
# Здесь основной код.

# Загружается RSS-лента
NewsFeed = feedparser.parse(RSS_URL)
# Создание класса озаглавливателя
titler = Titler()

# Формируем заголовки таблицы для вывода результата работы скрипта
columns = ['Исходный заголовок', 'Заголовок нейросети', 'Ссылка']
tab = PrettyTable(columns)
entryNum = 0

for entry in NewsFeed.entries:
  entryNum = entryNum + 1
  print("Processing page", entryNum, "of", len(NewsFeed.entries))
  # Извлекаем ссылку и исходное наименование статьи из RSS-ленты
  link = entry.link
  MailRuTitle = entry.title
  try:
    # Загружаем текст новости
    NewsText = getNewsText(link)
  except HTTPError as e:
    print(e.reason)
    break;
  # Запускаем модель для озаглавливания текста
  GeneratedTitle = titler.Title(NewsText)
  # Добавляем новую строку в таблицу
  tab.add_row([MailRuTitle, GeneratedTitle, link])

# Печатаем результат работы скрипта
print(tab)




Processing page 1 of 38
Processing page 2 of 38
Processing page 3 of 38
Processing page 4 of 38
Processing page 5 of 38
Processing page 6 of 38
Processing page 7 of 38
Processing page 8 of 38
Processing page 9 of 38
Processing page 10 of 38
Processing page 11 of 38
Processing page 12 of 38
Processing page 13 of 38
Processing page 14 of 38
Processing page 15 of 38
Processing page 16 of 38
Processing page 17 of 38
Processing page 18 of 38
Processing page 19 of 38
Processing page 20 of 38
Processing page 21 of 38
Processing page 22 of 38
Processing page 23 of 38
Processing page 24 of 38
Processing page 25 of 38
Processing page 26 of 38
Processing page 27 of 38
Processing page 28 of 38
Processing page 29 of 38
Processing page 30 of 38
Processing page 31 of 38
